In [1]:
import math
import random
import copy
import numpy as np
import pandas as pd
import time
from collections import deque

# =============================================================================
# --- DATA LOADING AND UTILITY FUNCTIONS ---
# =============================================================================
def charger_donnees_depuis_excel(chemin_fichier):
    print(f"--- Chargement des données depuis '{chemin_fichier}' ---")
    donnees = {}
    CONVERSION_MILE_TO_M = 1609.34
    df_puits = pd.read_excel(chemin_fichier, sheet_name='Puits')
    puits_data = {}
    pipe_costs_par_m = {4: round(358.69 / CONVERSION_MILE_TO_M, 5), 6: round(454.06 / CONVERSION_MILE_TO_M, 5)}
    for index, row in df_puits.iterrows():
        nom_puits, diam, debit_str = row['NomPuits'], row['DiametreFlowline_in'], row['Debit_kSm3_day']
        if diam not in pipe_costs_par_m: raise ValueError(f"Coût non défini pour le diamètre {diam}in.")
        puits_data[nom_puits] = {'x': float(row['X']), 'y': float(row['Y']), 'production_sm3d': float(str(debit_str).replace(',', '.')) * 1000, 'diametre_flowline_in': diam, 'CDF_i_dollar_par_m': pipe_costs_par_m[diam]}
    donnees['puits'] = puits_data
    print(f"Données des puits chargées: {len(puits_data)} puits.")
    df_manifolds = pd.read_excel(chemin_fichier, sheet_name='TypesManifold')
    donnees['manifolds'] = {row['TypeID']: {'capacite_puits': int(row['CapacitePuits']), 'cout_installation_dollar': float(row['CoutInstallation_dollar'])} for index, row in df_manifolds.iterrows()}
    print(f"Types de manifolds chargés: {len(donnees['manifolds'])} types.")
    df_trunklines = pd.read_excel(chemin_fichier, sheet_name='SpecsTrunkline')
    donnees['trunklines'] = sorted([{'diametre_in': int(r['Diametre_in']), 'cout_par_m_dollar': round(float(r['CoutParMile_dollar']) / CONVERSION_MILE_TO_M, 5), 'min_total_prod_sm3d': int(r['MinProd_Sm3d']), 'max_total_prod_sm3d': int(r['MaxProd_Sm3d'])} for i, r in df_trunklines.iterrows()], key=lambda x: x['min_total_prod_sm3d'])
    print(f"Spécifications de trunklines chargées: {len(donnees['trunklines'])} specs.")
    return donnees

def distance(p1, p2): return math.sqrt((p1.get('x', 0) - p2.get('x', 0))**2 + (p1.get('y', 0) - p2.get('y', 0))**2)

def get_trunkline_params(prod, specs):
    if not specs: return None, None
    for spec in specs:
        if spec['min_total_prod_sm3d'] <= prod <= spec['max_total_prod_sm3d']: return spec['diametre_in'], spec['cout_par_m_dollar']
    if prod > 0:
        s_specs = sorted(specs, key=lambda x: x['max_total_prod_sm3d'])
        if not s_specs: return None, None
        if prod > s_specs[-1]['max_total_prod_sm3d']: return s_specs[-1]['diametre_in'], s_specs[-1]['cout_par_m_dollar']
        if prod < s_specs[0]['min_total_prod_sm3d']: return s_specs[0]['diametre_in'], s_specs[0]['cout_par_m_dollar']
    return None, None

def get_zone_etude(puits):
    if not puits: return 0, 0, 0, 0
    all_x, all_y = [d['x'] for d in puits.values()], [d['y'] for d in puits.values()]
    return min(all_x), max(all_x), min(all_y), max(all_y)

def generer_sites_candidats_grille(x_min, x_max, y_min, y_max, pas, marge):
    sites = []
    for x in np.arange(x_min - marge, x_max + marge + pas / 2, pas):
        for y in np.arange(y_min - marge, y_max + marge + pas / 2, pas):
            sites.append({'x': round(x, 2), 'y': round(y, 2)})
    return sites

def calculer_cout_total_complet(sol, puits, m_types, t_specs, cpf_cost):
    if not sol or not sol.get('manifolds_ouverts') or not sol.get('cpf_location'): return float('inf')
    cost = cpf_cost
    active = {k: v for k, v in sol['manifolds_ouverts'].items() if v.get('puits_connectes')}
    for info in active.values(): cost += m_types[info['type_id']]['cout_installation_dollar']
    for p_id, m_id in sol['affectations_puits'].items():
        if m_id not in active: return float('inf')
        cost += distance(puits[p_id], active[m_id]) * puits[p_id]['CDF_i_dollar_par_m']
    cpf_loc = sol['cpf_location']
    for m_id, info in active.items():
        prod = sum(puits[p_id]['production_sm3d'] for p_id in info['puits_connectes'])
        d, c = get_trunkline_params(prod, t_specs)
        sol['manifolds_ouverts'][m_id]['total_production_sm3d'] = prod
        sol['manifolds_ouverts'][m_id]['DTj_in'] = d
        sol['manifolds_ouverts'][m_id]['CDTj_dollar_par_m'] = c
        if c: cost += distance(info, cpf_loc) * c
        elif prod > 0: return float('inf')
    return cost

def calculer_cout_phase1(sol, puits, m_types):
    if not sol or not sol.get('manifolds_ouverts'): return float('inf')
    cost = 0
    active = {k: v for k, v in sol['manifolds_ouverts'].items() if v.get('puits_connectes')}
    for info in active.values():
        if info['type_id'] not in m_types: return float('inf')
        cost += m_types[info['type_id']]['cout_installation_dollar']
    for p_id, m_id in sol['affectations_puits'].items():
        if m_id not in active: return float('inf')
        cost += distance(puits[p_id], active[m_id]) * puits[p_id]['CDF_i_dollar_par_m']
    return cost

# =============================================================================
# --- INITIAL SOLUTION HEURISTIC ---
# =============================================================================
def construire_solution_clustering(puits_data, types_manifold_data):
    clusters = {p_id: {'puits': {p_id}, 'center': p_data} for p_id, p_data in puits_data.items()}
    max_cap = max(info['capacite_puits'] for info in types_manifold_data.values())
    while True:
        if len(clusters) <= 1: break
        meilleure_fusion = {'dist': float('inf'), 'pair': None}
        cluster_ids = list(clusters.keys())
        for i in range(len(cluster_ids)):
            for j in range(i + 1, len(cluster_ids)):
                id1, id2 = cluster_ids[i], cluster_ids[j]
                c1, c2 = clusters[id1], clusters[id2]
                if len(c1['puits']) + len(c2['puits']) <= max_cap:
                    dist_clusters = distance(c1['center'], c2['center'])
                    if dist_clusters < meilleure_fusion['dist']:
                        meilleure_fusion = {'dist': dist_clusters, 'pair': (id1, id2)}
        if meilleure_fusion['pair'] is None: break
        id1, id2 = meilleure_fusion['pair']
        c1, c2 = clusters[id1], clusters[id2]
        nouveau_cluster_puits = c1['puits'].union(c2['puits'])
        nouveau_centre_x = sum(puits_data[p]['x'] for p in nouveau_cluster_puits) / len(nouveau_cluster_puits)
        nouveau_centre_y = sum(puits_data[p]['y'] for p in nouveau_cluster_puits) / len(nouveau_cluster_puits)
        nouveau_cluster = {'puits': nouveau_cluster_puits, 'center': {'x': nouveau_centre_x, 'y': nouveau_centre_y}}
        nouveau_id = f"{id1}-{id2}"
        del clusters[id1], clusters[id2]
        clusters[nouveau_id] = nouveau_cluster

    solution = {'manifolds_ouverts': {}, 'affectations_puits': {}}
    types_tries = sorted(types_manifold_data.items(), key=lambda item: item[1]['capacite_puits'])
    manifold_id_counter = 1
    for cluster in clusters.values():
        taille_cluster = len(cluster['puits'])
        type_choisi_id = next((tid for tid, tinfo in types_tries if taille_cluster <= tinfo['capacite_puits']), None)
        if type_choisi_id is None: return {'cout_total_phase1': float('inf')} 
        manifold_id = f"Manifold_{manifold_id_counter}"; manifold_id_counter += 1
        solution['manifolds_ouverts'][manifold_id] = {'type_id': type_choisi_id, 'x': cluster['center']['x'], 'y': cluster['center']['y'], 'puits_connectes': cluster['puits']}
        for p_id in cluster['puits']: solution['affectations_puits'][p_id] = manifold_id
    solution['cout_total_phase1'] = calculer_cout_phase1(solution, puits_data, types_manifold_data)
    return solution

def trouver_meilleure_config_manifolds(puits_data, manifold_types):
    print(f"--- Lancement de l'Heuristique de 'Clustering Itératif' ---")
    solution_p1 = construire_solution_clustering(puits_data, manifold_types)
    cout_p1 = solution_p1.get('cout_total_phase1', float('inf'))
    if cout_p1 == float('inf'): print("\n--- AVERTISSEMENT: La construction de la solution a échoué. ---")
    return solution_p1, cout_p1
    
def optimiser_cpf_pour_manifolds_fixes(solution_p1, puits_data, types_manifold_data, sites_candidats_cpf, trunkline_specs_data, cpf_cost):
    if not solution_p1 or not solution_p1.get('manifolds_ouverts'): return None, float('inf')
    meilleure_solution, meilleur_cout = None, float('inf')
    active_manifolds = {k: v for k, v in solution_p1['manifolds_ouverts'].items() if v.get('puits_connectes')}
    if not active_manifolds: return None, float('inf')
    solution_base = copy.deepcopy(solution_p1)
    solution_base['manifolds_ouverts'] = active_manifolds
    manifold_coords = {(round(m['x'],2), round(m['y'],2)) for m in active_manifolds.values()}
    for cpf_coords in sites_candidats_cpf:
        if (round(cpf_coords['x'],2), round(cpf_coords['y'],2)) in manifold_coords: continue
        solution_temp = copy.deepcopy(solution_base)
        solution_temp['cpf_location'] = cpf_coords
        cout_actuel = calculer_cout_total_complet(solution_temp, puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
        if cout_actuel < meilleur_cout:
            meilleur_cout, meilleure_solution = cout_actuel, copy.deepcopy(solution_temp)
            meilleure_solution['cout_total'] = meilleur_cout
    return meilleure_solution, meilleur_cout

def afficher_resultats_detailles(solution_finale, types_manifold_data, scenario_titre=""):
    print(f"\n\n--- RÉSULTAT FINAL DÉTAILLÉ ({scenario_titre}) ---")
    cout_total = solution_finale.get('cout_total', 0)
    cpf_loc = solution_finale.get('cpf_location', {'x': 'N/A', 'y': 'N/A'})
    print(f"Coût Total Optimal Estimé: ${cout_total:,.2f}")
    print(f"Localisation CPF: ({cpf_loc['x']:.2f}, {cpf_loc['y']:.2f})")
    manifolds_actifs = solution_finale.get('manifolds_ouverts', {})
    print(f"\nNombre de Manifolds Actifs: {len(manifolds_actifs)}")
    print("-" * 50)
    for m_id, m_data in sorted(manifolds_actifs.items()):
        type_id = m_data.get('type_id', 'N/A')
        capacite = types_manifold_data.get(type_id, {}).get('capacite_puits', 'N/A')
        print(f"  Manifold ID: {m_id}")
        print(f"    - Coordonnées: ({m_data.get('x', 0):.2f}, {m_data.get('y', 0):.2f})")
        print(f"    - Type: {type_id} (Capacité: {capacite})")
        diam_trunk = m_data.get('DTj_in', 'N/A')
        print(f"    - Trunkline vers CPF: Diamètre {diam_trunk} in")
        puits_connectes = sorted(list(m_data.get('puits_connectes', [])))
        print(f"    - Puits Connectés ({len(puits_connectes)}):")
        if puits_connectes:
            for i in range(0, len(puits_connectes), 6): print(f"      {'  '.join(puits_connectes[i:i+6])}")
        else: print("      Aucun")
        print("-" * 50)
        
# =============================================================================
# --- TABU SEARCH METAHEURISTIC ---
# =============================================================================
def generer_voisin_et_move_info(solution, types_manifold_data, sites_candidats, prob_moves):
    voisin = copy.deepcopy(solution)
    if not prob_moves: return None, None
    move_type = random.choices(list(prob_moves.keys()), weights=list(prob_moves.values()))[0]
    move_info = {'type': move_type}

    active_manifolds_dict = {m_id: m_data for m_id, m_data in voisin['manifolds_ouverts'].items() if m_data.get('puits_connectes')}
    active_manifold_ids = list(active_manifolds_dict.keys())
    
    if not active_manifold_ids: return None, None

    if move_type == 'reassign_well':
        if len(active_manifold_ids) < 2 or not voisin['affectations_puits']: return None, None
        p_id = random.choice(list(voisin['affectations_puits'].keys()))
        m_id_origine = voisin['affectations_puits'].get(p_id)
        if not m_id_origine: return None, None
        candidats = [m_id for m_id in active_manifold_ids if m_id != m_id_origine and len(active_manifolds_dict[m_id]['puits_connectes']) < types_manifold_data[active_manifolds_dict[m_id]['type_id']]['capacite_puits']]
        if not candidats: return None, None
        m_id_dest = random.choice(candidats)
        move_info.update({'p_id': p_id, 'm_from': m_id_origine, 'm_to': m_id_dest})
        voisin['affectations_puits'][p_id] = m_id_dest
        voisin['manifolds_ouverts'][m_id_origine]['puits_connectes'].remove(p_id)
        voisin['manifolds_ouverts'][m_id_dest]['puits_connectes'].add(p_id)

    elif move_type == 'swap_wells':
        if len(active_manifold_ids) < 2: return None, None
        m_id1, m_id2 = random.sample(active_manifold_ids, 2)
        puits_m1 = list(voisin['manifolds_ouverts'][m_id1]['puits_connectes'])
        puits_m2 = list(voisin['manifolds_ouverts'][m_id2]['puits_connectes'])
        if not puits_m1 or not puits_m2: return None, None
        p_id1_to_swap, p_id2_to_swap = random.choice(puits_m1), random.choice(puits_m2)
        voisin['manifolds_ouverts'][m_id1]['puits_connectes'].remove(p_id1_to_swap)
        voisin['manifolds_ouverts'][m_id1]['puits_connectes'].add(p_id2_to_swap)
        voisin['manifolds_ouverts'][m_id2]['puits_connectes'].remove(p_id2_to_swap)
        voisin['manifolds_ouverts'][m_id2]['puits_connectes'].add(p_id1_to_swap)
        voisin['affectations_puits'][p_id1_to_swap], voisin['affectations_puits'][p_id2_to_swap] = m_id2, m_id1
        move_info.update({'type': 'swap_wells', 'p_id1': p_id1_to_swap, 'p_id2': p_id2_to_swap})
        
    elif move_type == 'relocate_manifold':
        m_id = random.choice(active_manifold_ids)
        new_site = random.choice(sites_candidats)
        move_info.update({'m_id': m_id})
        voisin['manifolds_ouverts'][m_id]['x'], voisin['manifolds_ouverts'][m_id]['y'] = new_site['x'], new_site['y']

    elif move_type == 'relocate_cpf':
        new_site = random.choice(sites_candidats)
        move_info.update({'type': 'relocate_cpf'})
        voisin['cpf_location'] = new_site
    else: return None, None
    return voisin, move_info

def recherche_tabou(solution_initiale, puits_data, types_manifold_data, trunkline_specs_data, sites_candidats, cpf_cost, params, cpf_is_fixed=False):
    print(f"--- DÉBUT DE LA RECHERCHE TABOU (CPF Fixe: {cpf_is_fixed}) ---")
    start_time = time.time()
    
    MAX_ITER, TABU_TENURE, N_NEIGHBORS = params['MAX_ITERATIONS'], params['TABU_TENURE'], params['N_NEIGHBORS_TO_EVAL']
    
    # Adapt move probabilities based on the scenario
    prob_moves = params['PROB_MOVES'].copy()
    if cpf_is_fixed:
        if 'relocate_cpf' in prob_moves:
            del prob_moves['relocate_cpf']
        # Renormalize probabilities
        total_prob = sum(prob_moves.values())
        if total_prob > 0:
            prob_moves = {move: prob / total_prob for move, prob in prob_moves.items()}

    sol_actuelle = copy.deepcopy(solution_initiale)
    cout_actuel = calculer_cout_total_complet(copy.deepcopy(sol_actuelle), puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
    meilleure_sol, meilleur_cout = copy.deepcopy(sol_actuelle), cout_actuel
    tabu_list = deque(maxlen=TABU_TENURE)

    for i in range(MAX_ITER):
        meilleur_voisin_iter, meilleur_cout_voisin_iter, meilleur_move_info_iter = None, float('inf'), None
        for _ in range(N_NEIGHBORS):
            sol_voisine, move_info = generer_voisin_et_move_info(sol_actuelle, types_manifold_data, sites_candidats, prob_moves)
            if not move_info: continue
            
            tabu_attribute = None
            if move_info['type'] == 'reassign_well': tabu_attribute = ('well', move_info['p_id'])
            elif move_info['type'] == 'swap_wells': tabu_attribute = ('swap', frozenset([move_info['p_id1'], move_info['p_id2']]))
            elif move_info['type'] == 'relocate_manifold': tabu_attribute = ('manifold', move_info['m_id'])
            elif move_info['type'] == 'relocate_cpf': tabu_attribute = ('cpf',)
            
            cout_voisin = calculer_cout_total_complet(copy.deepcopy(sol_voisine), puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
            is_tabu = tabu_attribute in tabu_list
            aspiration_met = cout_voisin < meilleur_cout

            if (not is_tabu or aspiration_met) and (cout_voisin < meilleur_cout_voisin_iter):
                meilleur_voisin_iter, meilleur_cout_voisin_iter, meilleur_move_info_iter = sol_voisine, cout_voisin, move_info

        if meilleur_voisin_iter:
            sol_actuelle, cout_actuel = meilleur_voisin_iter, meilleur_cout_voisin_iter
            tabu_attribute_to_add = None 
            if meilleur_move_info_iter['type'] == 'reassign_well': tabu_attribute_to_add = ('well', meilleur_move_info_iter['p_id'])
            elif meilleur_move_info_iter['type'] == 'swap_wells': tabu_attribute_to_add = ('swap', frozenset([meilleur_move_info_iter['p_id1'], meilleur_move_info_iter['p_id2']]))
            elif meilleur_move_info_iter['type'] == 'relocate_manifold': tabu_attribute_to_add = ('manifold', meilleur_move_info_iter['m_id'])
            elif meilleur_move_info_iter['type'] == 'relocate_cpf': tabu_attribute_to_add = ('cpf',)
            if tabu_attribute_to_add: tabu_list.append(tabu_attribute_to_add)

            if cout_actuel < meilleur_cout:
                meilleur_cout, meilleure_sol = cout_actuel, copy.deepcopy(sol_actuelle)
                print(f"Iter {i+1}/{MAX_ITER}: Nouveau Meilleur Coût: ${meilleur_cout:,.2f}")
        if (i + 1) % 20 == 0: print(f"Iter {i+1}/{MAX_ITER}: Coût Actuel: ${cout_actuel:,.2f}, Meilleur Coût: ${meilleur_cout:,.2f}", end="\r")

    print(f"\n--- FIN DE LA RECHERCHE TABOU (Durée: {time.time() - start_time:.2f}s) ---")
    meilleure_sol['cout_total'] = meilleur_cout
    return meilleure_sol, meilleur_cout

# =============================================================================
# --- SCENARIO EXECUTION FUNCTIONS ---
# =============================================================================
def lancer_scenario_cpf_optimise(donnees, params):
    print("\n" + "="*60 + "\n--- SCÉNARIO 2 : CPF AVEC EMPLACEMENT OPTIMISÉ ---\n" + "="*60)
    PUITS, M_TYPES, T_SPECS = donnees['puits'], donnees['manifolds'], donnees['trunklines']

    print("--- PHASE 1: Construction de la solution initiale (Manifolds) ---")
    solution_p1, cout_p1 = trouver_meilleure_config_manifolds(PUITS, M_TYPES)
    if not solution_p1 or cout_p1 == float('inf'): print("\nÉCHEC DE LA PHASE 1"); return
    
    print(f"\n--- PHASE 2: Optimisation initiale du CPF ---")
    xmin, xmax, ymin, ymax = get_zone_etude(PUITS)
    sites_candidats = generer_sites_candidats_grille(xmin, xmax, ymin, ymax, params['PAS_GRILLE_MANIFOLD_M_VAL'], params['MARGE_POUR_GRILLE_M_VAL'])
    solution_initiale, cout_initial = optimiser_cpf_pour_manifolds_fixes(solution_p1, PUITS, M_TYPES, sites_candidats, T_SPECS, params['CPF_INSTALLATION_COST_DOLLAR_VAL'])
    if not solution_initiale: print("\nÉCHEC DE LA PHASE 2"); return
    
    print(f"Coût initial après Heuristique: ${cout_initial:,.2f}")
    
    print("\n--- PHASE 3: Amélioration par Recherche Tabou ---")
    solution_amelioree, cout_ameliore = recherche_tabou(
        solution_initiale, PUITS, M_TYPES, T_SPECS, sites_candidats, 
        params['CPF_INSTALLATION_COST_DOLLAR_VAL'], params, cpf_is_fixed=False)
        
    afficher_resultats_detailles(solution_amelioree, M_TYPES, "CPF Optimisé (Après Recherche Tabou)")
    print(f"\n--- RÉSUMÉ (CPF OPTIMISÉ) ---\nCoût Initial (Heuristique): ${cout_initial:,.2f}\nCoût Final (Recherche Tabou): ${cout_ameliore:,.2f}")
    print(f"Amélioration: {((cout_initial - cout_ameliore) / cout_initial * 100):.2f}%")

def lancer_scenario_cpf_fixe(donnees, params):
    print("\n" + "="*60 + "\n--- SCÉNARIO 1 : CPF AVEC EMPLACEMENT FIXE ---\n" + "="*60)
    PUITS, M_TYPES, T_SPECS = donnees['puits'], donnees['manifolds'], donnees['trunklines']
    print(f"Utilisation de l'emplacement CPF fixe: ({params['CPF_FIXE_X']}, {params['CPF_FIXE_Y']})")

    print("--- PHASE 1: Construction de la solution initiale (Manifolds) ---")
    solution_p1, cout_p1 = trouver_meilleure_config_manifolds(PUITS, M_TYPES)
    if not solution_p1 or cout_p1 == float('inf'): print("\nÉCHEC DE LA PHASE 1"); return

    solution_initiale = copy.deepcopy(solution_p1)
    solution_initiale['cpf_location'] = {'x': params['CPF_FIXE_X'], 'y': params['CPF_FIXE_Y']}
    cout_initial = calculer_cout_total_complet(copy.deepcopy(solution_initiale), PUITS, M_TYPES, T_SPECS, params['CPF_INSTALLATION_COST_DOLLAR_VAL'])
    solution_initiale['cout_total'] = cout_initial
    if cout_initial == float('inf'): print("\nERREUR: Impossible de calculer le coût initial."); return
    
    print(f"Coût initial après Heuristique: ${cout_initial:,.2f}")

    print("\n--- PHASE 2: Amélioration par Recherche Tabou ---")
    xmin, xmax, ymin, ymax = get_zone_etude(PUITS)
    sites_candidats_manifolds = generer_sites_candidats_grille(xmin, xmax, ymin, ymax, params['PAS_GRILLE_MANIFOLD_M_VAL'], params['MARGE_POUR_GRILLE_M_VAL'])
    solution_amelioree, cout_ameliore = recherche_tabou(
        solution_initiale, PUITS, M_TYPES, T_SPECS, sites_candidats_manifolds, 
        params['CPF_INSTALLATION_COST_DOLLAR_VAL'], params, cpf_is_fixed=True)

    afficher_resultats_detailles(solution_amelioree, M_TYPES, "CPF Fixe (Après Recherche Tabou)")
    print(f"\n--- RÉSUMÉ (CPF FIXE) ---\nCoût Initial (Heuristique): ${cout_initial:,.2f}\nCoût Final (Recherche Tabou): ${cout_ameliore:,.2f}")
    print(f"Amélioration: {((cout_initial - cout_ameliore) / cout_initial * 100):.2f}%")

# =============================================================================
# --- MAIN EXECUTION BLOCK ---
# =============================================================================
def main():
    """
    Fonction principale qui charge les données, définit les paramètres 
    et lance les scénarios d'optimisation.
    """
    try:
        # !!! ADAPTEZ LE CHEMIN VERS VOTRE FICHIER EXCEL ICI !!!
        fichier_excel = r'C:\Users\Dell\Downloads\donnees_probleme.xlsx'
        donnees = charger_donnees_depuis_excel(fichier_excel)
        
        # --- PARAMÈTRES COMMUNS ET POUR LA RECHERCHE TABOU ---
        PARAMS = {
            'CPF_INSTALLATION_COST_DOLLAR_VAL': 450000000.00,
            'MARGE_POUR_GRILLE_M_VAL': 5000.0,
            'PAS_GRILLE_MANIFOLD_M_VAL': 2000.0,
            
            # Coordonnées pour le scénario 1 (CPF Fixe)
            'CPF_FIXE_X': 449397.35,
            'CPF_FIXE_Y': 3048477.39,
            
            # Paramètres pour la Recherche Tabou
            'MAX_ITERATIONS': 600,
            'TABU_TENURE': 7,
            'N_NEIGHBORS_TO_EVAL': 50,
            'PROB_MOVES': {
                'reassign_well': 0.4,
                'swap_wells': 0.3,
                'relocate_manifold': 0.2,
                'relocate_cpf': 0.1
            }
        }
        
        # --- Lancement des deux scénarios ---
        # Vous pouvez commenter l'une ou l'autre ligne pour ne tester qu'un scénario.
        lancer_scenario_cpf_fixe(donnees, PARAMS)
        lancer_scenario_cpf_optimise(donnees, PARAMS)
        
    except FileNotFoundError:
        print(f"\nERREUR CRITIQUE: Le fichier '{fichier_excel}' n'a pas été trouvé.")
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"\nUNE ERREUR EST SURVENUE: {e}")

# Point d'entrée du script
if __name__ == "__main__":
    main()

--- Chargement des données depuis 'C:\Users\Dell\Downloads\donnees_probleme.xlsx' ---
Données des puits chargées: 143 puits.
Types de manifolds chargés: 3 types.
Spécifications de trunklines chargées: 13 specs.

--- SCÉNARIO 1 : CPF AVEC EMPLACEMENT FIXE ---
Utilisation de l'emplacement CPF fixe: (449397.35, 3048477.39)
--- PHASE 1: Construction de la solution initiale (Manifolds) ---
--- Lancement de l'Heuristique de 'Clustering Itératif' ---
Coût initial après Heuristique: $531,357,652.83

--- PHASE 2: Amélioration par Recherche Tabou ---
--- DÉBUT DE LA RECHERCHE TABOU (CPF Fixe: True) ---
Iter 1/600: Nouveau Meilleur Coût: $531,353,359.34
Iter 2/600: Nouveau Meilleur Coût: $531,353,281.51
Iter 3/600: Nouveau Meilleur Coût: $531,347,047.96
Iter 5/600: Nouveau Meilleur Coût: $531,346,469.75
Iter 6/600: Nouveau Meilleur Coût: $531,334,619.67
Iter 7/600: Nouveau Meilleur Coût: $531,333,643.61
Iter 8/600: Nouveau Meilleur Coût: $531,327,996.49
Iter 20/600: Nouveau Meilleur Coût: $531,32